## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd

data = pd.read_csv('/datasets/data.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


#### Вывод

Изучение общей информации дало следующие результаты:  
- Датасет имеет 21525 строки
- В двух стобцах (days_employed, total_income) имеются пропущенные значения
- Названия некоторых столбцов не совсем удобные (dob_years, income_type, total_income)  

Изучение первых 10 строк таблицы дало следующие результаты:
- Встречаются отрицательные значения в столбце days_employed, хотя отрицательного количества дней быть не может
- В столбце education строки находятся в разных регистрах

### Шаг 2. Предобработка данных

#### Изменение названий стобцов

Изменим названия столбцов, заменив dob_years на client_age, income_type на work_type, total_income на month_income

In [3]:
data.set_axis(
    ['children',
     'days_employed',
     'client_age',
     'education',
     'education_id',
     'family_status',
     'family_status_id',
     'gender',
     'work_type',
     'debt',
     'month_income',
     'purpose'], axis='columns', inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
client_age          21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
work_type           21525 non-null object
debt                21525 non-null int64
month_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


#### Обработка значений в столбце days_employed

В данном столбце встречаются подозрительные значения двух типов:
- Все положительные значения слишком большие
- Все "нормальные" значения -- отрицательные

Необходимо:
- Слишком большие положительные значения заменить на среднее арифметическое отрицательных значений
- Все значения сделать положительными

In [4]:
negative_mean = data.loc[data['days_employed'] < 0, 'days_employed'].mean()
data.loc[data['days_employed'] > 0, 'days_employed'] = negative_mean
data['days_employed'] = data['days_employed'].apply(abs)
data.head(5)

,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,work_type,debt,month_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,2353.015932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Вычисляем среднее значение столбца days_employed

In [5]:
days_employed_mean = data.loc[data['days_employed'].notnull(), 'days_employed'].mean()

#### Обработка значений в столбце children

Выведем все значения, встречающиеся в данном столбце

In [6]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Логичная картина: чем больше детей у человека, тем меньше таких людей встречается. Поэтому тот факт, что людей с 20 детьми больше, чем людей с 4 или 5 детьми, немного смущает.
Также очевидно, что отрицательного количества детей быть не может.

Решения:
- Вероятнее всего отрицатльное количество детей (-1) - это ошибка ввода. Поэтому значения -1 можно поменять на 1
- Скорее всего 20 - это тоже ошибка ввода. Поэтому значения 20 меняем на 2.

Количество выбивающихся значений невелико, поэтому, если были приняты ошибочные решения, это крайне слабо отразится на результате.

In [7]:
data['children'] = data['children'].replace([20, -1], [2, 1]) # Значения 20 меняются на 2; -1 на 1
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

#### Обработка значений в столбце client_age

Выведем все значения, встречающие в данном столбце.

In [8]:
data['client_age'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: client_age, dtype: int64

"Подозрительные" данные: клиенты с возрастом 0, 2 и 1. Посмотрим на этих клиентов внимательнее.

In [9]:
data[data['client_age'] == 1].head(5)

,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,work_type,debt,month_income,purpose


In [10]:
data[data['client_age'] == 2].head(5)

,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,work_type,debt,month_income,purpose


Стало ясно, что клиенты с возрастом 1 или 2 года - мусор в данных, от которого надо избавиться

Удаляем строки, в которых возраст клиента равен 2 или 1

In [11]:
data.drop(data[(data['client_age'] == 2) | (data['client_age'] == 1)].index, inplace=True)

Остались клиенты с нулевым возрастом.

In [12]:
data[data['client_age'] == 0].head(10)

,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,work_type,debt,month_income,purpose
99,0,2353.015932,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,2353.015932,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,0,934.654854,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,2353.015932,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,5043.219890,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,NaN,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,NaN,жилье
1898,0,2353.015932,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


Во многих языках программирования 0 - значение по умолчанию для целочисленных типов. Скорее всего в данной таблице 0 - это пропущенное значение. 
Возраст - количественный параметр, у которого есть четкие рамки. Поэтому пропущенные значения можно заменить средним арифметическим всех возрастов.

Возраст клиентов с нулевым возрастом приравниваем среднему возрасту клиентов с ненулевым возрастом

In [13]:
data.loc[data['client_age'] == 0, 'client_age'] = int(data.loc[data['client_age'] != 0, 'client_age'].mean())

#### Обработка значений в столбце education

Выведем все значения, встречающие в данном столбце.

In [14]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Видно, что одни и те же строки присутствуют в разных регистрах. Переведем все строки в нижний регистр и повторим вывод.

In [15]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

#### Обработка значений в столбце family_status

Выведем все значения, встречающие в данном столбце.

In [16]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Все значения нас удовлетворяют. Необходимо только перевести все строки в нижний регистр.

In [17]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

#### Обработка значений в столбце gender

Выведем все значения, встречающие в данном столбце.

In [18]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Посмотрим на клиента с подозрительным полом "XNA"

In [19]:
data[data['gender'] == 'XNA']

,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,work_type,debt,month_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Ничего примечательного не обнаружено. С большей вероятностью пол этого клиента - женский. Поменяем пол этого клиента.

In [20]:
data.loc[data['gender'] == 'XNA', 'gender'] = 'F'

#### Обработка пропусков


Проверим предположение о том, что если данные пропущены в одном из столбцов days_employed и month_income, то они пропущены и в другом из этих столбцов. Иначе говоря, пропуски в этих столбцах совпадают.

Вычисляем количество строк, в которых одновременно пропущены значения в столбцах days_employed и month_income

In [21]:
data.loc[(data['days_employed'].isna()) & (data['month_income'].isna())].shape[0]

2174

Вычисляем количество пропусков в каждом из этих двух столбцов одновременно

In [22]:
data.loc[:, ['days_employed', 'month_income']].isna().sum()

days_employed    2174
month_income     2174
dtype: int64

Видно, что пропуски в столбцах days_employed и month_income совпадают. Это значит, что у таких клиентов одновременно отсутствует информация о трудовом стаже и ежемесячному доходу. Можно предположить, что эти клиенты просто не указали информацию о своей работе.  
Это могли быть и люди, которые никогда не работали. Но встречаются клиенты, у которых пропущены значения в столбцах days_employed и month_income, но в столбце work_type указано "сотрудник". Поэтому это предположение неверно.
Обработаем пропуски.

Пропуски в столбце days_employed случайны. При этом данный столбец содержит количественные значения.
Значения данного столбца лежат в определенных пределах, поэтому отсутствующие значения можно заменить на среднее арифметическое. Используем значение, полученное выше.

In [23]:
data['days_employed'] = data['days_employed'].fillna(value=days_employed_mean)
data['days_employed'].isnull().sum()

0

Пропуски в столбце month_income также случайны и содержат количественные значения.
Однако можно предположить, что ежемесячный доход (month_income) зависит от типа занятости (work_type). При этом доход не имеет границ, поэтому для рассчета среднего стоит использовать медианное значение.
Рассчитаем медианное значение заработка для каждого типа занятости.

In [24]:
data['month_income'] = data.groupby('work_type')['month_income'].apply(lambda x: x.fillna(x.median()))
data['month_income'].isnull().sum()

0

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
client_age          21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
work_type           21525 non-null object
debt                21525 non-null int64
month_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


##### Вывод

Было обнаружено, что пропуски в столбцах days_employed и month_income совпадают. Скорее всего, это произошло из-за того, что эти клиенты либо не указали, либо не сообщили свои данные о работе.

Результат замены пропусков:
- В столбце days_employed пропуски были заменены на среднее арифметическое значение.
- В столбце month_income пропуски были заменены на медианные значения. Значения были рассчитаны для каждого типа занятости.  

#### Замена типа данных

Почти все столбцы содержат значения подходящих типов. Однако столбец days_employed содержит нецелочисленные значения, что не очень удобно. Будет правильнее считать количество полных дней. Для этого необходимо у каждого значения отбросить дробную часть. Воспользуемся методом int().

In [26]:
data['days_employed'] = data['days_employed'].apply(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
client_age          21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
work_type           21525 non-null object
debt                21525 non-null int64
month_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


##### Вывод

Замена типа данных потребовалась только в одном столбце - days_employed.  
Вряд-ли подобные вещественные значения вводил человек. Скорее всего это делала какая-то система, а в таблицу записывался результат деления одного числа на другое.  
Была произведена замена типа с вещественного, на целочисленный.

#### Обработка дубликатов

Узнаем общее количество дубликатов и посмотрим на сами дубликаты

In [27]:
data.duplicated().sum()

71

In [28]:
data[data.duplicated(keep=False)].sort_values(by='client_age').head(30)

,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,work_type,debt,month_income,purpose
20297,1,2353,23,среднее,1,гражданский брак,1,F,сотрудник,0,142594.396847,сыграть свадьбу
8853,1,2353,23,среднее,1,гражданский брак,1,F,сотрудник,0,142594.396847,сыграть свадьбу
15892,0,2353,23,среднее,1,не женат / не замужем,4,F,сотрудник,0,142594.396847,сделка с подержанным автомобилем
19321,0,2353,23,среднее,1,не женат / не замужем,4,F,сотрудник,0,142594.396847,сделка с подержанным автомобилем
3452,0,2353,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594.396847,покупка жилой недвижимости
18328,0,2353,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594.396847,покупка жилой недвижимости
4216,0,2353,30,среднее,1,женат / замужем,0,M,сотрудник,0,142594.396847,строительство жилой недвижимости
6312,0,2353,30,среднее,1,женат / замужем,0,M,сотрудник,0,142594.396847,строительство жилой недвижимости
18349,1,2353,30,высшее,0,женат / замужем,0,F,госслужащий,0,150447.935283,покупка жилья для семьи
8629,1,2353,30,высшее,0,женат / замужем,0,F,сотрудник,0,142594.396847,покупка коммерческой недвижимости


Повторяющиеся значения в столбцах days_employed и month_income - это результат заполнения пропусков.  
Показателем того, что мы наблюдаем действительно дубликаты, могут служить значения столбцов client_age и purpose. Данные в этих столбцах достаточно уникальны, поэтому случайные совпадения маловероятны.
Избавляемся от дубликатов.

In [29]:
data = data.drop_duplicates().reset_index(drop = True)
data.duplicated().sum()

0

##### Вывод

Зрительный анализ совпадающих строк показал, что эти совпадения - действительно дубликаты. Никаких зависимостей выявлено не было. Всего было удалено 72 дубликата.

#### Лемматизация

Изучив значения, встречающиеся во всех столбцах, пришли к выводу, что только для столбца purpose необходимо провести лемматизацию. Это позволит ответить на один поставленных вопросов в шаге 3.  
Для того, чтобы правильно провести лемматизацию, посмотрим на встречающиеся значения в столбце purpose.

In [30]:
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Посмотрим как часто встречается отдельно взятая лемма

In [31]:
from pymystem3 import Mystem
from collections import Counter

m = Mystem()
lemmas = m.lemmatize(' '.join(data['purpose']))
print(Counter(lemmas))

Counter({' ': 55023, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


Все эти значения можно распределить на 4 группы:
- свадьба  
- недвижимость
- автомобиль
- образование  

Более мелкие группы выделять не будем. Заменим значения столбца purpose на новые, в соответствии с обозначенными группами.

In [32]:
def lemmatize_purpose(purpose):
    lemmas = m.lemmatize(purpose)
    if 'жилье' in lemmas or 'недвижимость' in lemmas: 
        return 'недвижимость'
    elif 'свадьба' in lemmas: 
        return 'свадьба'
    elif 'образование' in lemmas: 
        return 'образование'
    else: 
        return 'автомобиль'
    
data['purpose'] = data['purpose'].apply(lemmatize_purpose)

Проверим не остались ли неизмененные значения:

In [33]:
data['purpose'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose, dtype: int64

Все значения были изменены, значит лемматизация прошла успешно.

##### Вывод

Лемматизация столбца purpose была необходима для ответа на один из поставленных вопросов.  
Все значения столбца были разделены на 4 группы. После лемматизации не осталось ни одного значения не отнесенного к какой-либо группе.  
Получилась следующая картина: чем "дороже" цель, тем чаще люди берут для неё кредит.

#### Категоризация данных

Для ответа на один из поставленных в третьем шаге вопросов необходимо провести категоризацию значений в столбце month_income. Категоризация позволит группировать данные по уровню дохода.    
Сначала узнаем диапазон значений данного столбца.

In [34]:
min_month_income = data['month_income'].min()

In [35]:
max_month_income = data['month_income'].max()

In [36]:
print(f'Диапазон значений: {min_month_income:.2f} - {max_month_income:.2f}')

Диапазон значений: 20667.26 - 2265604.03


Составим список категорий и укажем диапазон для каждой категории:
- низкий 20 000 - 60 000
- средний 60 000 - 120 000
- высокий 120 000 - 250 000
- очень высокий 250 000 - 400 000
- крайне высокий > 400 000  

Создадим новый столбец month_income_level, который будет содержать категории уровня дохода.

In [37]:
data['month_income_level'] = pd.cut(
    data['month_income'],                                                                     # Столбец для разбивки
    [
        data['month_income'].min(), 60000, 120000, 250000, 400000, data['month_income'].max() # Границы диапазонов
    ], 
    labels=['низкий', 'средний', 'высокий', 'очень высокий', 'крайне высокий']                # Названия диапазонов
)

In [38]:
data['month_income_level'].value_counts()

высокий           11410
средний            6425
очень высокий      2284
низкий              805
крайне высокий      529
Name: month_income_level, dtype: int64

##### Вывод

Категоризация столбца month_income_level была необходима для ответа на один из поставленных вопросов.  
Были составлены 5 категорий. Для каждой категории был определен диапазон.  
Клиентов с высоким и средним уровнем дохода оказалось больше всего, что достаточно логично. У клиентов с таким уровнем дохода достаточно высокий доход, чтобы взять кредит, но не достаточно высокий, чтобы его не приходилось брать.  

Категоризацию можно было бы выполнить и для столбца client_age, но для ответа на поставленные вопросы это не потребовалось.

### Шаг 3. Ответьте на вопросы

При ответах на все представленные вопросы в качестве меры надежности клиента будет использовано среднее значение столбца debt.  

Данный столбец принимает значения 0 и 1, где:
- 0 - клиент не имел задолженность по кредиту
- 1 - клиент имел задолженность по кредиту  

Поэтому, чем меньше среднее значение, тем клиент надежнее.

#### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [39]:
data.groupby('children')['debt'].mean().sort_values()

children
5    0.000000
0    0.075438
3    0.081818
1    0.091658
2    0.094925
4    0.097561
Name: debt, dtype: float64

Для удобства добавим информацию о том сколько детей и скольких клиентов.

In [40]:
data['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Также выведем средний месячный доход, сгруппированный по количеству детей.

In [41]:
data.groupby('children')['month_income'].mean().sort_values()

children
0    163052.609653
4    166361.224093
5    167337.417929
1    169041.267136
2    169623.321109
3    179460.802355
Name: month_income, dtype: float64

##### Вывод

Общая тенденция: чем больше детей, тем клиент менее надежный.  
Клиентов с пятью детьми слишком мало, поэтому их учитывать не будем.  
Также немного выбиваются клиенты с 3 детьми. Если посмотреть на средний доход клиентов с 3 детьми, то видно, что такие клиенты в среднем зарабатывают больше, чем остальные клиенты. Поэтому чуть б*о*льшая надежность клиентов с 3 детьми обусловлена чуть б*о*льшим доходом.  
Вывод: зависимость есть: чем больше детей, тем менее надежный клиент.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [42]:
data.groupby('family_status')['debt'].mean().sort_values()

family_status
вдовец / вдова           0.065693
в разводе                0.071130
женат / замужем          0.075452
гражданский брак         0.093471
не женат / не замужем    0.097509
Name: debt, dtype: float64

##### Вывод

Установить зависимость между надежностью клиента и его семейным положением не удалось.  

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [43]:
data.groupby('month_income_level')['debt'].mean().sort_values()

month_income_level
крайне высокий    0.058601
низкий            0.059627
очень высокий     0.071366
средний           0.083580
высокий           0.084224
Name: debt, dtype: float64

##### Вывод

Установить зависимость между уровнем дохода и возвратом кредита в срок не удалось.

#### Как разные цели кредита влияют на его возврат в срок?

In [44]:
data.groupby('purpose')['debt'].mean().sort_values()

purpose
недвижимость    0.072334
свадьба         0.080034
образование     0.092200
автомобиль      0.093590
Name: debt, dtype: float64

##### Вывод

Зависимость не обнаружена. Логично только то, что покупка автомобиля - самая ненадежная цель получения кредита. Ведь автомобиль - достаточно дорогая вещь, которая ещё и требует постоянных вложений.

### Шаг 4. Общий вывод

Исследование данных показало, что зависимости надёжности клиента от целей кредита или семейного положения нет.  
Была обнаружена зависимость от количества детей. Посмотрим как на надёжность влияет сочетание количества детей и уровня дохода.

In [45]:
data.pivot_table(index=['month_income_level'], columns='children', values='debt', aggfunc='mean').loc[:'высокий', :'2']

children,0,1,2
month_income_level,,,
низкий,0.050000,0.060403,0.121212
средний,0.077447,0.095817,0.100486
высокий,0.079146,0.095463,0.090186


В данной таблице были отсечены ячейки, соответствующие очень высокому уровню дохода и количеству детей больше 2. Данные в тех ячейках были непоказательны.  
В оставшихся ячейках видно:
- С увеличением количества детей падает надежность клиентов с любым уровнем дохода
- Клиент с более низким уровнем дохода и меньшим количеством детей более надежный, чем клиент с более высоким уровнем дохода и б*о*льшим количеством детей  

Отвечая на поставленные вопросы можно сделать **главный вывод**: для выяснения надёжности клиентов стоит в первую очередь стоит обратить внимание на сочетание уровня дохода и количества детей клиента.  

Во время предобработки данных были заполнены пропуски, удалены дубликаты и очевидно ошибочные данные, отформатированы значения в некоторых столбцах. Также была проведена лемматизация и категоризация значений в столбцах month_income и purpose. Подробнее:
- В столбце children были обработанны нереальные и нелогичные значения -1 и 20
- В столбце days_employed:
    - Слишком большие положительные значения заменены на среднее арифметическое отрицательных значений
    - Все значения были сделаны положительными
    - Пропуски заполнены средним арифметическим значением
- Нулевой возраст клиентов заменен на средний возраст клиентов с ненулевым возрастом
- Значения столбца education переведены в нижний регистр
- Значения столбца family_status переведены в нижний регистр
- Нереальный пол одного клиента сменен на самый часто встречающийся
- Заполнены пропуски в столбце month_income
- Проведена лемматизация целей кредита (столбец purpose)
- Проведена категоризация уровня дохода (столбец month_income_level)

В ходе работы сделаны промежуточные выводы:
- Больше всего заёмщиков со средним и высоким уровнем дохода (доход от 60 000 до 250 000 в месяц)
- Чаще всего берут кредит на операции с недвижимостью
- Чаще всего берут кредит люди среднего возраста
- Люди, которые берут кредит, чаще всего женаты или находятся замужем  

Рекомендации:  
- В столбцах children, days_employed, client_age, gender и month_income встречались пропуски или сигнализирующие о пропусках значения. Вероятнее всего, в системах, в которых происходит заполнение этих данных, существует возможность эти данные не указывать. В таком случае эти поля лучше заполнять значением None или NaN. Так легче всего будет обрабатывать пропуски.  
- В столбце education были дублирующиеся значения в разных регистрах. Чтобы избежать ошибок стоит переводить все значения к единому регистру (лучше нижнему).  
- В столбце days_employed все нормальные значения были отрицаительными. Вероятно какой-то этап внесения этих данных происходит не совсем корректно. Стоит это проверить.  
- Судя по исходным данным в столбце purpose, клиент вводит / называет цель кредита вручную. При лемматизации сложно правильно выделить узкие группы целей кредита, поэтому приходится выделять более общие. Предлагается создать список из более узких групп целей кредита и предлагать клиенту выбирать из него. Так будет легче оценить влияние цели кредита на надёжность клиента.
